In [1]:
import numpy as np
from scipy import misc
import pprint as pp
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.losses import hinge
from keras import backend as Keras

Using TensorFlow backend.


In [2]:
import sys
sys.path.insert(0, '/home/albert/github/DenseNet/')
import densenet

### Train Network

In [3]:
CAFFE_ROOT = '/home/albert/caffe/'
img_dir = os.listdir(CAFFE_ROOT + 'data/market-1501/bounding_box_train')

In [4]:
train_img_paths = []
labels = []
labels_one_hot = []

for f in img_dir:
    if f[-4:] == '.jpg':
        idt = int(f[0:f.index('_')])
        if not any(idt == l for l in labels):
            labels.append(idt)
            
labels.sort()

for f in img_dir:
    if f[-4:] == '.jpg':
        idt = int(f[0:f.index('_')])
        path = CAFFE_ROOT + 'data/market-1501/bounding_box_train/' + f
        train_img_paths.append(path)
        one_hot = np.zeros(751).tolist()
        one_hot[labels.index(idt)] = 1
        labels_one_hot.append(one_hot)

In [5]:
def output_batch_generator(train_files, labels_one_hot, batch_size=50):
    while True:
        batch = []
        l = []
        p_choice = np.random.choice(len(train_files), batch_size, replace=False)
        for p in p_choice:
            img = cv2.resize(misc.imread(train_files[p]), (224, 224)).astype(np.float64)
            batch.append(img.tolist())
            l.append(labels_one_hot[p])
        yield(np.array(batch), np.array(l))

In [ ]:
generator = output_batch_generator(train_img_paths, labels_one_hot)
batch, _ = generator.next()

In [58]:
def evaluate_rank(net, rank, all_embeddings, all_identities, test_iter=1000):
    correct = 0
    f_choice = np.random.choice(range(len(train_img_paths)), np.minimum(test_iter, len(train_img_paths)), replace=False)
    for f in f_choice:
        query_img = cv2.resize(misc.imread(train_img_paths[f]), (224,224))
        query_embedding = net.predict(query_img.reshape(1, 224, 224, 3))
        distance_vectors = np.squeeze(np.abs(all_embeddings - query_embedding))
#         print(distance_vectors)
        distance = np.sum(distance_vectors, axis=1)
        top_inds = distance.argsort()[:rank+1]
        output_classes = np.array(all_identities)[top_inds].astype(np.uint16)
        
#         pp.pprint(zip(distance[top_inds], np.array(all_identities)[top_inds].astype(np.uint16)))
#         print(int(all_identities[f]))
        
        i = 0
        for c in output_classes:
            if c == int(all_identities[f]):
                i += 1
        if i > 1:
            correct += 1
    return float(correct)/test_iter

In [69]:
# https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes
i = 0
# for epochs in range(10, 20, 10):
image_dim = (224, 224, 3)
base_model = densenet.DenseNetImageNet121(image_dim)
print(len(base_model.layers))

Weights for the model were loaded successfully
427


In [70]:
base_model.outputs = [base_model.layers[-2].output]
base_model.layers[-1].outbound_nodes = []

In [71]:
x = base_model.outputs[0]
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
predictions = Dense(751)(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss=hinge, optimizer=Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6),
              metrics=['acc'])

In [72]:
for layer in model.layers:
    layer.trainable = True

In [73]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_241 (Conv2D)              (None, 112, 112, 64)  9408        input_3[0][0]                    
____________________________________________________________________________________________________
batch_normalization_245 (BatchNo (None, 112, 112, 64)  256         conv2d_241[0][0]                 
____________________________________________________________________________________________________
activation_245 (Activation)      (None, 112, 112, 64)  0           batch_normalization_245[0][0]    
___________________________________________________________________________________________

In [21]:
epochs = 100
model.fit_generator(output_batch_generator(train_img_paths, labels_one_hot, batch_size=12), steps_per_epoch=50, epochs=epochs)

Epoch 1/100
50/50 [==============================] - 46s - loss: 1.2269 - acc: 0.0033    
Epoch 2/100
50/50 [==============================] - 38s - loss: 1.1240 - acc: 0.0067    
Epoch 3/100
50/50 [==============================] - 39s - loss: 1.0780 - acc: 0.0033    
Epoch 4/100
50/50 [==============================] - 38s - loss: 1.0536 - acc: 0.0033    
Epoch 5/100
50/50 [==============================] - 38s - loss: 1.0396 - acc: 0.0033    
Epoch 6/100
50/50 [==============================] - 38s - loss: 1.0307 - acc: 0.0017    
Epoch 7/100
50/50 [==============================] - 38s - loss: 1.0245 - acc: 0.0000e+00    
Epoch 8/100
50/50 [==============================] - 38s - loss: 1.0201 - acc: 0.0000e+00    
Epoch 9/100
50/50 [==============================] - 38s - loss: 1.0169 - acc: 0.0000e+00    
Epoch 10/100
50/50 [==============================] - 38s - loss: 1.0144 - acc: 0.0000e+00    
Epoch 11/100
50/50 [==============================] - 39s - loss: 1.0125 - acc: 0.0

50/50 [==============================] - 38s - loss: 0.9990 - acc: 0.0033    
Epoch 91/100
50/50 [==============================] - 38s - loss: 0.9989 - acc: 0.0017    
Epoch 92/100
50/50 [==============================] - 38s - loss: 0.9989 - acc: 0.0033    
Epoch 93/100
50/50 [==============================] - 37s - loss: 0.9991 - acc: 0.0000e+00    
Epoch 94/100
50/50 [==============================] - 38s - loss: 0.9990 - acc: 0.0017    
Epoch 95/100
50/50 [==============================] - 37s - loss: 0.9989 - acc: 0.0017    
Epoch 96/100
50/50 [==============================] - 38s - loss: 0.9989 - acc: 0.0000e+00    
Epoch 97/100
50/50 [==============================] - 38s - loss: 0.9988 - acc: 0.0017    
Epoch 98/100
50/50 [==============================] - 37s - loss: 0.9988 - acc: 0.0017    
Epoch 99/100
50/50 [==============================] - 38s - loss: 0.9988 - acc: 0.0033    
Epoch 100/100
50/50 [==============================] - 38s - loss: 0.9989 - acc: 0.0000e+00    

Load model if needed. Continue training net, but now with adaptive learning rate.

In [22]:
model.save('/home/albert/github/tensorflow/densenet_classifier_%d.h5' % epochs)

In [38]:
layer_name = 'dense_5'
intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
all_embeddings = []
all_identities = []
for f in range(len(train_img_paths)):
    img = cv2.resize(misc.imread(train_img_paths[f]), (224,224))
    predict = intermediate_layer_model.predict(img.reshape(1,224,224,3))
    all_embeddings.append(predict)
    all_identities.append(labels_one_hot[f].index(1))

In [56]:
net = intermediate_layer_model
f=100
query_img = cv2.resize(misc.imread(train_img_paths[f]), (224,224))
query_embedding = net.predict(query_img.reshape(1, 224, 224, 3))

In [57]:
print(query_embedding)

[[ 0.03815887 -0.08697419  0.03913601 ...,  0.06300279 -0.09919606
   0.04945709]]


In [55]:
print all_embeddings

[array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279,
        -0.09919606,  0.04945709]], dtype=float32), array([[ 0.03815887, -0.08697419,  0.03913601, ...,  0.06300279

In [ ]:
distance_vectors = np.squeeze(np.abs(all_embeddings - query_embedding))
print(distance_vectors)
distance = np.sum(distance_vectors, axis=1)
top_inds = distance.argsort()[:rank+1]
output_classes = np.array(all_identities)[top_inds].astype(np.uint16)

In [59]:
i = 0
for x in range(3):
    score = evaluate_rank(intermediate_layer_model, 20, all_embeddings, all_identities, test_iter=1000)
    print score
i += 1

0.0
0.0
0.0


In [26]:
# np.save('/home/albert/github/tensorflow/rank20_learn_all.npy', np.array(score_arr))

In [27]:
print score_arr

[[0.136, 0.153, 0.148]]


https://keras.io/getting-started/faq/: use load_model to reinstantiate model.

Train all, lr_decay=1e-6, use log1p

### Evaluate Performance

In [ ]:
p = 7
query_img = cv2.resize(misc.imread(train_files[p][2]), (224,224))
query_embedding = trinet.predict(model.predict(query_img.reshape(1, 224, 224, 3)))

In [ ]:
distance_vectors = np.squeeze(np.abs(all_embeddings - query_embedding))
distance = np.sum(distance_vectors, axis=1)

In [ ]:
top_inds = distance.argsort()[:20]
output_classes = np.array(all_identities)[top_inds]
pp.pprint(zip(distance[top_inds], np.array(all_identities)[top_inds].astype(np.uint16)))